In [2]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 13
year = 2022

data = get_data(day=day, year=year)

In [3]:
data = """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]"""

In [4]:
from functools import cmp_to_key

packets = [[eval(x) for x in l.split('\n')] for l in data.split("\n\n")]

cmp = lambda a,b: (a < b) - (a > b)

def is_ordered(l, r):
    l_i, r_i = isinstance(l, int), isinstance(r, int)
    if l_i and r_i:
        return cmp(l, r)
    if l_i: l = [l]
    if r_i: r = [r]
    
    arr = (is_ordered(a, b) for a,b in zip(l,r))
    return next((o for o in arr if o != 0), cmp(len(l), len(r)))
    
print("Part 1:", sum(i+1 for i,p in enumerate(packets) if is_ordered(*p) > 0))

dividers = [[[2]], [[6]]]
sorted_packets = sorted([p for pair in packets for p in pair] + dividers,
                        key=cmp_to_key(is_ordered), reverse=True)

d1, d2 = [i+1 for i,p in enumerate(sorted_packets) if p in dividers]
print("Part 2:", d1*d2)


# print("Using Python 3.10 (match/case):")

# from functools import cmp_to_key

# packets = [[eval(x) for x in l.split('\n')] for l in data.split("\n\n")]

# cmp = lambda a,b: (a < b) - (a > b)

# def is_ordered(l, r):
#     match l, r:
#         case int(), int():
#             return cmp(l, r)
#         case int(), list():
#             l = [l]
#         case list(), int():
#             r = [r]
#     arr = (is_ordered(a, b) for a,b in zip(l,r))
#     return next((o for o in arr if o != 0), cmp(len(l), len(r)))
    
# print("Part 1:", sum(i+1 for i,p in enumerate(packets) if is_ordered(*p) > 0))

# dividers = [[[2]], [[6]]]
# sorted_packets = sorted([p for pair in packets for p in pair] + dividers,
#                         key=cmp_to_key(is_ordered), reverse=True)

# d1, d2 = [i+1 for i,p in enumerate(sorted_packets) if p in dividers]
# print("Part 2:", d1*d2)


Part 1: 13
Part 2: 140


Part 1: 13
Part 2: 140


In [1060]:
len(grid)

6314

['D', 'C']